In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def collate_fn(batch):
  batch = list(filter(lambda x: x is not None, batch))
  return torch.utils.data.dataloader.default_collate(batch)

class selfData:
  def __init__(self, img_path, target_path, transforms = None):
    with open(target_path, 'r') as f:
      lines = f.readlines()
      self.img_list = [os.path.join(img_path, i.split()[0]) for i in lines]
      self.label_list = [i.split()[1] for i in lines]
      self.transforms = transforms

  def __getitem__(self, index):
    try:
      img_path = self.img_list[index]
      img = Image.open(img_path)
      img = self.transforms(img)
      label = self.label_list[index]
    except:
      return None
    return img, label

  def __len__(self):
    return len(self.label_list)

In [ ]:
class mAlexNet(nn.Module):
  def __init__(self, num_classes = 2):
    super(mAlexNet, self).__init__()
    self.input_channel = 3
    self.num_output = num_classes
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=self.input_channel, out_channels= 16, kernel_size= 11, stride= 4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels= 16, out_channels= 20, kernel_size= 5, stride= 1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels= 20, out_channels= 30, kernel_size= 3, stride= 1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2)
    )
    self.layer4 = nn.Sequential(
        nn.Linear(30*3*3, out_features=48),
        #nn.Linear(30, out_features=48),
        nn.ReLU(inplace=True)
    )
    self.layer5 = nn.Sequential(
        nn.Linear(in_features=48, out_features=2)
    )

  def forward(self, x):
    x = self.layer3(self.layer2(self.layer1(x)))
    x = x.view(x.size(0), -1)
    x = self.layer5(self.layer4(x))
    m = nn.Softmax(dim = 1)
    x = m(x)
    return x

In [ ]:
def train(epoch, img_path, target_path, transforms, net, criterion):
  train_dataset = selfData(img_path, target_path, transforms)
  train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers = 0,drop_last= False, collate_fn=collate_fn)
  print('Training begins...')
  for ep in range(epoch):  
    learning_rate = 0.01
    if ep >= 12:
      learning_rate = 0.0025
    elif ep >= 6:
      learning_rate = 0.005
    running_loss = 0.0
    
    for i, data in enumerate(train_loader,1):
      inputs, labels = data
      labels = list(map(int, labels))
      labels = torch.Tensor(labels)
      if torch.cuda.is_available():
        device = torch.device("cuda:0")
        inputs = inputs.to(device)
        labels = labels.to(device)
      optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, labels.long())
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
  print('Finished Training.')

In [ ]:
def test(img_path, target_path, transforms, net):
  print("\nTesting starts now...")
  test_dataset = selfData(img_path, target_path, transforms)
  test_loader = DataLoader(test_dataset, batch_size = 100, shuffle = True, num_workers = 0, collate_fn=collate_fn)
  correct = 0
  total = 0
  item = 1
  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      # print("Testing on batch {}".format(item))
      labels = list(map(int, labels))
      labels = torch.Tensor(labels)
      if torch.cuda.is_available():
        device = torch.device("cuda:0")
        images = images.to(device)
        labels = labels.to(device)
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      item += 1
      accuracy = (correct/total)
  return accuracy

In [ ]:
epochs = 18
net = mAlexNet().to(device)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tf = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),  # normalize to [0, 1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]) 

In [ ]:
train_img = 'PKLot/PKLotSegmented'
train_lab = ['splits/PKLot/UFPR04_train.txt','splits/PKLot/UFPR05_train.txt', 'splits/PKLot/PUC_train.txt']
test_img = 'PKLot/PKLotSegmented'
test_lab = ['splits/PKLot/UFPR04_test.txt','splits/PKLot/UFPR05_test.txt', 'splits/PKLot/PUC_test.txt']

for train_set in train_lab:
    train(epochs, train_img, train_set, tf, net, criterion)
    PATH = './trained.pth'
    torch.save(net.state_dict(), PATH)
    net = mAlexNet().to(device)
    net.load_state_dict(torch.load(PATH))
    for test_set in test_lab:      
        accuracy = test(test_img, test_set, tf, net)
        print("Training on '{}' and testing on '{}': {:.3f}.\n".format(train_set.split('.')[0], test_set.split('.')[0], accuracy))
print("Experiments ended.")

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/PKLot/UFPR04_train' and testing on 'splits/PKLot/UFPR04_test': 0.960.


Testing starts now...
Training on 'splits/PKLot/UFPR04_train' and testing on 'splits/PKLot/UFPR05_test': 0.799.


Testing starts now...
Training on 'splits/PKLot/UFPR04_train' and testing on 'splits/PKLot/PUC_test': 0.930.

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/PKLot/UFPR05_train' and testing on 'splits/PKLot/UFPR04_test': 0.800.


Testing starts now...
Training on 'splits/PKLot/UFPR05_train' and testing on 'splits/PKLot/UFPR05_test': 0.976.


Testing starts now...
Training on 'splits/PKLot/UFPR05_train' and testing on 'splits/PKLot/PUC_test': 0.901.

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/PKLot/PUC_train' and testing on 'splits/PKLot/UFPR04_test': 0.956.


Testing starts now...
Training on 'splits/PKLot/PUC_train' and testing on 'splits/PKLot/UFPR05_t

In [ ]:
train_img = 'CNRPark-Patches-150x150'
train_lab = ['splits/CNRParkAB/odd.txt','splits/CNRParkAB/even.txt']
test_img = 'CNRPark-Patches-150x150'
test_lab = ['splits/CNRParkAB/even.txt','splits/CNRParkAB/odd.txt']

for train_set, test_set in zip(train_lab, test_lab):
    train(epochs, train_img, train_set, tf, net, criterion)
    PATH = './trained.pth'
    torch.save(net.state_dict(), PATH)
    net = mAlexNet().to(device)
    net.load_state_dict(torch.load(PATH))
    accuracy = test(test_img, test_set, tf, net)
    print("Training on '{}' and testing on '{}': {:.3f}.\n".format(train_set.split('.')[0], test_set.split('.')[0], accuracy))
print("Experiments ended.")

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/CNRParkAB/odd' and testing on 'splits/CNRParkAB/even': 0.919.

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/CNRParkAB/even' and testing on 'splits/CNRParkAB/odd': 0.924.

Experiments ended.


In [ ]:
train_img = 'CNRPark-EXT/PATCHES'
train_lab = ['splits/CNRPark-EXT/sunny.txt','splits/CNRPark-EXT/overcast.txt','splits/CNRPark-EXT/rainy.txt']
test_lab = ['splits/CNRPark-EXT/sunny.txt','splits/CNRPark-EXT/overcast.txt','splits/CNRPark-EXT/rainy.txt','splits/PKLot/val.txt']

for train_set in train_lab:
    train(epochs, train_img, train_set, tf, net, criterion)
    PATH = './trained.pth'
    torch.save(net.state_dict(), PATH)
    net = mAlexNet().to(device)
    net.load_state_dict(torch.load(PATH))
    for test_set in test_lab:
        if test_set == train_set:
            accuracy = 'none'
            print("Skip to next test.")
        elif test_set == 'splits/PKLot/val.txt':
            test_img = 'PKLot/PKLotSegmented'
            accuracy = test(test_img, test_set, tf, net)
            print("Training on '{}' and testing on '{}': {:.3f}.\n".format(train_set.split('.')[0], test_set.split('.')[0], accuracy))
        else:
            test_img = 'CNRPark-EXT/PATCHES/'
            accuracy = test(test_img, test_set, tf, net)
            print("Training on '{}' and testing on '{}': {:.3f}.\n".format(train_set.split('.')[0], test_set.split('.')[0], accuracy))
print("Experiments ended.")

Training begins...
Finished Training.
Skip to next test.

Testing starts now...
Training on 'splits/CNRPark-EXT/sunny' and testing on 'splits/CNRPark-EXT/overcast': 0.946.


Testing starts now...
Training on 'splits/CNRPark-EXT/sunny' and testing on 'splits/CNRPark-EXT/rainy': 0.912.


Testing starts now...
Training on 'splits/CNRPark-EXT/sunny' and testing on 'splits/PKLot/val': 0.759.

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/CNRPark-EXT/overcast' and testing on 'splits/CNRPark-EXT/sunny': 0.917.

Skip to next test.

Testing starts now...
Training on 'splits/CNRPark-EXT/overcast' and testing on 'splits/CNRPark-EXT/rainy': 0.920.


Testing starts now...
Training on 'splits/CNRPark-EXT/overcast' and testing on 'splits/PKLot/val': 0.709.

Training begins...
Finished Training.

Testing starts now...
Training on 'splits/CNRPark-EXT/rainy' and testing on 'splits/CNRPark-EXT/sunny': 0.914.


Testing starts now...
Training on 'splits/CNRPark-EXT/rainy'